In [14]:

# Step 1 - Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
from splinter import Browser
import requests
import datetime as dt
import time

# NASA Mars News

In [15]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
#Assign the text to variables that you can reference later.

## Assign path
import os
if os.name=="nt":
    executable_path = {'executable_path': './chromedriver.exe'}
else:
    executable_path = {"executable_path": "/chromedriver"}

In [16]:
browser = Browser('chrome', **executable_path, headless=True)

In [17]:
## URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [22]:

# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Examine the results, then determine element that contains sought info
info = soup.find_all('div', class_="list_text")

#Create a container for the data that will be scraped
news_title = []
news_p = []

# Loop through returned results
for x in info:
    title = x.find('a').text.strip()
    paragraph = x.find( class_='article_teaser_body').text.strip()
    
    news_title.append(title)
    news_p.append(paragraph)
        
    print('----------')
    print(title)
    print(paragraph)
        
print('-----------')
print(f"News title count: {len(news_title)}, and new paragraph count: {len(news_p)} on this page")

----------
How NASA's Mars Helicopter Will Reach the Red Planet's Surface
The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.
----------
The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan.
----------
NASA to Hold Mars 2020 Perseverance Rover Launch Briefing
Learn more about the agency's next Red Planet mission during a live event on June 17.
----------
While Stargazing on Mars, NASA's Curiosity Rover Spots Earth and Venus
This new portrait of the Red Planet's neighbors was taken during a time when there's more dust in the air on Mars.
----------
NASA's Mars Rover Drivers Need Your Help
Using an online tool to label Martian terrain types, you can train an artificial intelligence algorithm that could i

In [23]:
#Save to CSV file
news_mars = pd.DataFrame({"News Title": news_title,"News Paragraph" : news_p
})

news_mars.head()

,News Title,News Paragraph
0,How NASA's Mars Helicopter Will Reach the Red ...,The small craft will seek to prove that powere...
1,The Launch Is Approaching for NASA's Next Mars...,The Red Planet's surface has been visited by e...
2,NASA to Hold Mars 2020 Perseverance Rover Laun...,Learn more about the agency's next Red Planet ...
3,"While Stargazing on Mars, NASA's Curiosity Rov...",This new portrait of the Red Planet's neighbor...
4,NASA's Mars Rover Drivers Need Your Help,Using an online tool to label Martian terrain ...


In [26]:
news_mars.to_csv("Saved Output/news_mars.csv")

# JPL Mars Space Images - Featured Image

In [28]:
# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars#submit'
browser.visit(url)

In [29]:

# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Examine the results, then determine element that contains sought info
info = soup.find_all('div', class_="img")


#Assign the url string to a variable called featured_image_url
#Create a container for the data that will be scraped
featured_image_url= []

# Loop through returned results
for x in info:

    image = x.find('img')
    image_loc = image['src']
    
    print('https://www.jpl.nasa.gov' + image_loc)  
    
    featured_image_url.append('https://www.jpl.nasa.gov' + image_loc)
    print('--------')
        
    
print('--------')
print(f"Count of images in the webpage {len(featured_image_url)}")

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23960-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23959-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23958-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23957-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23956-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23950-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23949-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23948-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23900-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23947-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23899-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23946-640x350.jpg
-

In [30]:
#Inspect data
print (featured_image_url)

['https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23960-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23959-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23958-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23957-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23956-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23950-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23949-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23948-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23900-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23947-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23899-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23946-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23897

In [31]:
#import dataframe to convert the list to df
from pandas import DataFrame

In [32]:
featured_df = DataFrame (featured_image_url, columns=['Featured Mars Images'])
print(featured_df)

                                 Featured Mars Images
0   https://www.jpl.nasa.gov/spaceimages/images/wa...
1   https://www.jpl.nasa.gov/spaceimages/images/wa...
2   https://www.jpl.nasa.gov/spaceimages/images/wa...
3   https://www.jpl.nasa.gov/spaceimages/images/wa...
4   https://www.jpl.nasa.gov/spaceimages/images/wa...
5   https://www.jpl.nasa.gov/spaceimages/images/wa...
6   https://www.jpl.nasa.gov/spaceimages/images/wa...
7   https://www.jpl.nasa.gov/spaceimages/images/wa...
8   https://www.jpl.nasa.gov/spaceimages/images/wa...
9   https://www.jpl.nasa.gov/spaceimages/images/wa...
10  https://www.jpl.nasa.gov/spaceimages/images/wa...
11  https://www.jpl.nasa.gov/spaceimages/images/wa...
12  https://www.jpl.nasa.gov/spaceimages/images/wa...
13  https://www.jpl.nasa.gov/spaceimages/images/wa...
14  https://www.jpl.nasa.gov/spaceimages/images/wa...
15  https://www.jpl.nasa.gov/spaceimages/images/wa...
16  https://www.jpl.nasa.gov/spaceimages/images/wa...
17  https://www.jpl.nasa.gov

In [34]:
#Save the df as CSV file 
featured_df.to_csv("Saved Output/featured_image_url.csv", index=False)

# Mars Weather

In [35]:
# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [37]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Examine the results, then determine element that contains sought info
info = soup.find_all('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")

#Create a container for the data that will be scraped
mars_weather = []
   
for x in info: 
    weather = x.text.strip()
    if (weather!="" and weather[0]=='I'):
        mars_weather.append(weather)
        print(weather)
        print('---------')
     
    print(f"The count of tweet text for Mars's weather is {len(mars_weather)}")

The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weath

In [39]:
#Save the tweet text for the weather report as a variable called mars_weather
mars_weather_df = DataFrame (mars_weather, columns=['Mars Weather Tweets'])
print(mars_weather_df)

                                 Mars Weather Tweets
0  InSight sol 566 (2020-06-29) low -88.4ºC (-127...
1  It's our birthday! On this day in 1976, a sign...
2  InSight sol 565 (2020-06-28) low -89.6ºC (-129...
3  InSight sol 564 (2020-06-28) low -89.3ºC (-128...


In [40]:
#Save the df as CSV file 
mars_weather_df.to_csv("Saved Output/mars_weather.csv", index=False)


# Mars Facts

In [41]:
# URL of page to be scraped
url = 'https://space-facts.com/mars/'

In [42]:
#Read and inspect the data
mars_facts = pd.read_html(url)
mars_facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [43]:
#Convert list to dataframe
mars_facts_df = mars_facts[0]
mars_facts_df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [44]:
#Rename the column titles
mars_facts_df = mars_facts_df.rename(columns={0:"Unit", 1:"Measure"})
mars_facts_df

,Unit,Measure
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [45]:
#Save the dataframe to html
mars_facts_df.to_html('Saved Output/mars_facts.html')

# Mars Hemispheres

In [118]:
# URL of page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [119]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [120]:
# Examine the results, then determine element that contains sought info
info = soup.find_all('div', class_="item")
info

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiapa

In [121]:
#Create a container for the data that will be scraped
hemisphere_details = []
   
#Loop through the returned results for Hemisphere name    
for x in info:

    description = x.find('div', class_="description")
    h3 = description.find('h3').text.strip()
    
    a = x.find('a')
    href = a['href']
    
    hemisphere_name_url = 'https://astrogeology.usgs.gov' + href
    print(hemisphere_name_url)
    
    hemisphere_details.append('https://astrogeology.usgs.gov' + href)
    print('--------')   


https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
--------
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
--------
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
--------
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
--------


In [122]:
hemisphere_details

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [123]:
import requests
import time


hemisph_images = {}
   
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all elements that contain articles information
infos = soup.find_all('div', class_="item")

for info in infos:

    description = info.find('div', class_="description")
    h3 = description.find('h3').text.strip()
    
    link = info.find('a')
    href = link['href']
    
    url_full = 'https://astrogeology.usgs.gov' + href
    print(url_full)
    
    browser.visit(url_full)
    time.sleep(2)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    info_image = soup.find_all('img', class_="wide-image")
    
    href_full =  info_image[0]['src']
    
    hemisph_images.update( {h3 : 'https://astrogeology.usgs.gov' + href_full} )
    print('title:',h3)
    print('image_url:','https://astrogeology.usgs.gov' + href_full)
    print('-----------')
        
print(f"The {len(hemisph_images)} images and their links")

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
title: Cerberus Hemisphere Enhanced
image_url: https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
-----------
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
title: Schiaparelli Hemisphere Enhanced
image_url: https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
-----------
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
title: Syrtis Major Hemisphere Enhanced
image_url: https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
-----------
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
title: Valles Marineris Hemisphere Enhanced
image_url: https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
-----------
The 